In [1]:
import pandas as pd
import praw
import datetime        
from time import sleep
import random
import time



## Building

In [2]:
import praw
import pandas as pd
from tqdm import tqdm

In [3]:
reddit = praw.Reddit(
    client_id='fKSnwxwrgn7DXbSDq8oHnw',         # Your app's client ID
    client_secret='xtz8CGEQjdB4SNbM9eHTp82eP0sIdQ', # Your app's client secret
    user_agent='Copenhagen University Data Collection Course, analyzing comment civility, Oliver',       # A descriptive user agent
    username='OkMinute',    # Your Reddit username
    password='Fodbold7'     # Your Reddit password
)

Version 7.7.1 of praw is outdated. Version 7.8.1 was released Friday October 25, 2024.


In [4]:
def get_top_posts(subreddit_list, limit=1000, sort_by='top', time_filter='all'):
    """
    Fetch top posts from a list of subreddits and return as a DataFrame.
    
    Parameters:
        subreddit_list (list): List of subreddit names.
        limit (int): Number of posts to fetch per subreddit.
        sort_by (str): Sorting method ('top', 'hot', 'new', 'rising').
        time_filter (str): Time filter for sorting ('all', 'day', 'hour', 'month', 'week', 'year'). Only for 'top'.
    
    Returns:
        pd.DataFrame: DataFrame containing post data.
    """
    
    post_list = []
    
    for subreddit_name in subreddit_list:
        try:
            subreddit = reddit.subreddit(subreddit_name)

            if sort_by == 'top':
                posts = subreddit.top(limit=limit, time_filter=time_filter)
            elif sort_by == 'hot':
                posts = subreddit.hot(limit=limit)
            elif sort_by == 'new':
                posts = subreddit.new(limit=limit)
            elif sort_by == 'rising':
                posts = subreddit.rising(limit=limit)
            else:
                raise ValueError("Invalid sort_by value. Use 'top', 'hot', 'new', or 'rising'.")
            
            for post in posts:
                post_list.append({
                    'subreddit': subreddit_name,
                    'title': post.title,
                    'score': post.score,
                    'id': post.id,
                    'url': post.url,
                    'num_comments': post.num_comments,
                    'created_utc': post.created_utc,
                    'selftext': post.selftext,
                    'author': post.author.name if post.author else None,
                    'upvote_ratio': post.upvote_ratio,
                    'permalink': post.permalink,
                    'domain': post.domain,
                    'flair_text': post.link_flair_text,
                    'is_self': post.is_self,
                    'view_count': post.view_count,
                    'media': post.media
                })
        
        except Exception as e:
            print(f"Error fetching posts from r/{subreddit_name}: {e}. Skipping...")

    return pd.DataFrame(post_list)


In [5]:
def get_post_comments(post, max_comments=50):
    """
    Fetch up to max_comments for a given post object.
    
    Parameters:
        post (praw.models.Submission): The Reddit post object.
        max_comments (int): Maximum number of comments to fetch.
    
    Returns:
        pd.DataFrame: DataFrame containing comment data.
    """
    post.comments.replace_more(limit=0)  # Flatten all comments

    comment_list = []
    for i, comment in enumerate(post.comments.list()):
        if i >= max_comments:
            break

        comment_list.append({
            'subreddit': post.subreddit.display_name,
            'post_id': post.id,
            'post_title': post.title,
            'comment_id': comment.id,
            'author': comment.author.name if comment.author else None,
            'score': comment.score,
            'created_utc': comment.created_utc,
            'body': comment.body,
            'parent_id': comment.parent_id,
            'depth': comment.depth,
            'controversiality': comment.controversiality
        })

    return pd.DataFrame(comment_list)

import time

def get_all_comments(df_posts):
    """
    Loop over posts DataFrame and collect all comments for each post with progress tracking.
    
    Parameters:
        df_posts (pd.DataFrame): DataFrame containing posts.
    
    Returns:
        pd.DataFrame: DataFrame containing all collected comments.
    """
    
    all_comments = []
    for _, post in tqdm(df_posts.iterrows(), total=len(df_posts), desc="Fetching Comments"):

        # Check API rate limits
        limits = reddit.auth.limits
        if limits["remaining"] < 300:
            print(f"Tokens low ({limits['remaining']} remaining). Waiting 5 minutes to avoid rate limit...")
            time.sleep(300)  # Wait for 5 minutes before resuming
        
        submission = reddit.submission(id=post['id'])
        comments_df = get_post_comments(submission)
        all_comments.append(comments_df)
        
    return pd.concat(all_comments, ignore_index=True) if all_comments else pd.DataFrame()



In [6]:
# Example usage
subreddits = [
    "nutrition",
    "keto",
    "healthyfood",
    "intermittentfasting",
    "fasting",
    "PlantBasedDiet",
    "Volumeeating",
    "EatCheapAndHealthy",
    "CICO",
    "vegetarian"
]



df_posts = get_top_posts(subreddits, limit=1000, sort_by='top', time_filter='all')

# Display DataFrame
df_posts.head()

# Collect all comments for all posts with progress tracking
df_comments = get_all_comments(df_posts)



Fetching Comments:  30%|██▉       | 2982/9949 [1:01:59<1:07:24,  1.72it/s]

Tokens low (299.0 remaining). Waiting 5 minutes to avoid rate limit...


Fetching Comments:  56%|█████▌    | 5572/9949 [2:02:52<46:02,  1.58it/s]    

Tokens low (299.0 remaining). Waiting 5 minutes to avoid rate limit...


Fetching Comments:  68%|██████▊   | 6800/9949 [2:22:27<34:08,  1.54it/s]    

Tokens low (299.0 remaining). Waiting 5 minutes to avoid rate limit...


Fetching Comments:  90%|████████▉ | 8924/9949 [3:12:56<14:35,  1.17it/s]   

Tokens low (299.0 remaining). Waiting 5 minutes to avoid rate limit...


Fetching Comments: 100%|██████████| 9949/9949 [3:34:38<00:00,  1.29s/it]   


In [7]:
df_comments.to_csv("b_conventional_comments.csv", index=False, encoding="utf-8")

In [8]:
df_posts.to_csv("a_conventional_posts.csv")

In [13]:
reddit.auth.limits

{'remaining': 999.0, 'reset_timestamp': 1740491401.68855, 'used': 1}